## Import Modules/ Libs for Data Clean-up

In [1]:
import pandas as pd
import csv
import datetime as dt
from datetime import datetime
import numpy as np

## Import SQL-Alchemy for SQL-Lite Data Load

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session
from sqlalchemy import func
import pathlib

## Data Clean-up Process

### DATABASE RAW_DF

In [ ]:
# define path to csv file
path="Resources_hle\IMDb_movies.csv"

# convert csv to pandas df
raw_df = pd.read_csv(path, encoding="UTF-8", dtype={'usa_gross_income': "string", "worlwide_gross_income" : "string", "budget": "string"})

1. choose all cols to keep (netflix id, imdb id, title, ddmmyy, dd, mo,yr,duration, avg_vote, votes, reviews_from_users	reviews_from_critics, budget,usa_gross_icome, ww income, coutry, type, genre, actor, description)
[[""]]
2. break the date published col into ye, mo, dd
3. combine reviews
4. rename columns
5. load into SQL-pgadmin4
6. report
       a. where data found, how
       b. transforming process
       c. load SQL, pic of SQL

In [ ]:
# review what imported
raw_df

In [ ]:
# describe the data    
raw_df.describe()

In [ ]:
# extract day, month, year from the date_published columns
raw_df["date_published"] = pd.to_datetime(raw_df['date_published'])
raw_df['day'], raw_df['month'], raw_df['year']  = raw_df['date_published'].dt.day, raw_df['date_published'].dt.month, raw_df['date_published'].dt.year

In [ ]:
# print out current columns with template to create a dictionary for columns rename 
for col in raw_df.columns:
    print(f'"{col}" : "",')

In [ ]:
# define what will be renamed
cols = {
"imdb_title_id" : "id",
"production_company" : "prod_co",
"avg_vote" : "user_rating",
"votes" : "vote_num",
"metascore" : "web_rating",
    }

# process new col names
raw_df.rename(columns=cols, inplace=True)
raw_df

In [ ]:
# convert to numbers
raw_df['reviews_from_users'] = pd.to_numeric(raw_df["reviews_from_users"], downcast="integer")
raw_df['reviews_from_critics'] = pd.to_numeric(raw_df["reviews_from_critics"],  downcast="integer")
raw_df['web_rating'] = pd.to_numeric(raw_df["web_rating"],  downcast="integer")

In [ ]:
# create a new col combining reviews from users and critics
raw_df['total_reviews'] = raw_df['reviews_from_users'] + raw_df['reviews_from_critics']

In [ ]:
# # replace all NaN with empty
raw_df.fillna('', inplace=True)
raw_df

In [ ]:
# print out all current column names
for col in raw_df.columns:
    print(f'"{col}",')

1. choose all cols to keep (netflix id, imdb id, title, ddmmyy, dd, mo,yr,duration, avg_vote, votes, reviews_from_users	reviews_from_critics, budget,usa_gross_icome, ww income, coutry, type, genre, actor, description)
[[""]]
2. break the date published col into ye, mo, dd
3. combine reviews
4. rename columns
5. load into SQL-pgadmin4
6. report
       a. where data found, how
       b. transforming process
       c. load SQL, pic of SQL

### DATABASE PROCESSED_DF

In [ ]:
 del processed_df

In [ ]:
# # filter out un-needed columns and re-arrange columns
try:
    del processed_df

except Exception:
    pass

processed_df = raw_df #[[
# "id",
# "title",
# "day",
# "month",
# "year",
# "date_published",
# "user_rating",
# "web_rating",
# "vote_num",
# "reviews_from_users",
# "reviews_from_critics",
# "total_reviews",
# "budget",
# "usa_gross_income",
# "worlwide_gross_income",
# "duration",
# "country",
# "language",
# "genre",
# "director",
# "writer",
# "prod_co",
# "actors",
# "description"
# ]]

In [ ]:
processed_df

In [ ]:
# processed_df.loc[:,'worlwide_gross_income'] = processed_df.loc[:, 'worlwide_gross_income'].str.replace("$", "USD ")
# processed_df.loc[:,'worlwide_gross_income'] = processed_df.loc[:, 'worlwide_gross_income'].str.replace(",", "")
# processed_df

In [ ]:
# SPLIT COLUMNS
# processed_df['wgi_currency'], processed_df['worlwide_gross_income'] = processed_df['worlwide_gross_income'].str.split(' ', 1).str


In [ ]:
# processed_df.loc[:,'worlwide_gross_income'] = processed_df.loc[:, 'worlwide_gross_income'].str.replace("$", "USD ")
# processed_df.loc[:,'worlwide_gross_income'] = processed_df.loc[:, 'worlwide_gross_income'].str.replace(",", "")

In [ ]:
# def convt_currency (df_name, col_name):
    
#     print (f">> Processing Column: '{col_name}'>>\n{50*('-')}")
    
#     # replace $ for USD and remove all "blanks"
#     df_name.loc[:,col_name] = df_name.loc[:, col_name].str.replace("$", "USD ")
#     df_name.loc[:,col_name] = df_name.loc[:, col_name].str.replace(",", "")

#     # split currency and value into two cols
#     for row in df_name[col_name]:
#         try:
#             new_col = f'{col_name}_currency'
# #             print(new_col)
#             df_name[new_col], df_name[col_name] = df_name[col_name].str.split(' ', 1)

#         except ValueError:
#             pass
    
#     df_name.fillna('', inplace=True)
        
#     return print(">> DONE!")

In [ ]:
processed_df[['worlwide_gross_income', 'usa_gross_income', 'budget']]

In [ ]:
def conv_currency (df_name, col_name):
    
    print (f"{50*('-')}\n >> Processing Column: '{col_name}'")
    
    # replace $ for USD and remove all "blanks"
    df_name.loc[:,col_name] = df_name.loc[:, col_name].str.replace("$", "USD ")
    df_name.loc[:,col_name] = df_name.loc[:, col_name].str.replace(",", "")

    # split currency and value into two cols
    
    try:
        new_col = f'{col_name}_currency'
        print(f'\t>>> Creating new column named: "{new_col}"')
        df_name[new_col], df_name[col_name] = df_name[col_name].str.split(' ', 1).str

    except ValueError:
        pass
    
    df_name.fillna('', inplace=True)
    
    return print(" >> DONE!")

In [ ]:
col_to_covert = ['worlwide_gross_income', 'usa_gross_income', 'budget']

In [ ]:
for ea_col in col_to_covert:
    conv_currency(processed_df, ea_col)
processed_df[['worlwide_gross_income', 'usa_gross_income', 'budget']]


In [ ]:
processed_df[['worlwide_gross_income', 'usa_gross_income', 'budget']].to_csv("exported_draft.csv")

In [ ]:
# conv_currency(processed_df, 'worlwide_gross_income')
# processed_df

In [ ]:
# processed_df[["worlwide_gross_income", 'currency']].iloc[156]

In [ ]:
# processed_df['worlwide_gross_income'].dtypes

In [ ]:
# for index, row in processed_df.iterrows():
#     row['id'] = row['id'].replace("tt", "") 
#     row['worlwide_gross_income'] =  row['worlwide_gross_income'].replace("$", "")
#     row['worlwide_gross_income'] =  row['worlwide_gross_income'].replace(" ", "")
# processed_df

### SQL-Alchemy to load clean data from pandas DataFrame to SQL Lite DataBase

In [ ]:
# create declarative base
Base = declarative_base()

# check current table available in the Base - should be nothing at this point
Base.metadata.tables


In [ ]:
# folder name that will store the sql-lite database
fol_name = "Exp_SqlLiteDb"
if pathlib.Path(fol_name).exists():
    print(f' >> Folder "{fol_name}" already exists!\n >> No new folder was created ...')
    pass
else:
    ! mkdir Exp_SqlLiteDb
    print(f'Successfully created folder "{fol_name}"')

In [ ]:
# # folder name that will store the sql-lite database
# database_name = "Exp_SqlLiteDb"
# if pathlib.Path(fol_name).exists():
#     print(f' >> Folder "{fol_name}" already exists!\n >> No new folder was created ...')
#     pass
# else:
#     ! mkdir Exp_SqlLiteDb
#     print(f'Successfully created folder "{fol_name}"')

In [ ]:
sqlLite_db_path = "Exp_SqlLiteDb\movie.db"
engine = create_engine(f"sqlite:///{sqlLite_db_path}")
conn = engine.connect()

In [ ]:
# print out list of cols for class creation
i=0
for col in processed_df.columns:
    print(f'{col} = Column()')
    i+=1
print(f'\n{("-")*50}\n>> There are total {i} columns in the current data frame.')

In [ ]:
# column types in pd_df
processed_df.dtypes

In [ ]:
id                               object
title                            object
day                               int64
month                             int64
year                              int64
date_published           datetime64[ns]
user_rating                     float64
web_rating                      float64
vote_num                          int64
reviews_from_users              float64
reviews_from_critics            float64
total_reviews                   float64
budget                           object
usa_gross_income                 object
worlwide_gross_income            object
duration                          int64
country                          object
language                         object
genre                            object
director                         object
writer                           object
prod co                          object
actors                           object
description                      object

In [ ]:
class Movie(Base):
    table_name = input("Please name your table.")
    __tablename__ = table_name
    id = Column(Integer, primary_key=True)
    title = Column(String(255))
    day = Column(Integer)
    month = Column(Integer)
    year = Column(Integer)
    date_published = Column(String(15))
    user_rating = Column(REAL)
    web_rating = Column(REAL)
    vote_num = Column(Integer)
    reviews_from_users = Column(Integer)
    reviews_from_critics = Column(Integer)
    total_reviews = Column(Integer)
    budget = Column(String(255))
    usa_gross_income = Column(String(255))
    worlwide_gross_income = Column(Interger)
    duration = Column(Interger)
    country = Column(String(50)
    language = Column(String(50))
    genre = Column(String(50))
    director = Column(String(50))
    writer = Column(String(50))
    prod_co = Column(String(50))
    actors = Column(String(50))
    description = Column(String(500))


In [ ]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
Base.metadata.create_all(engine)

In [ ]:
# current in memory tables
Base.metadata.tables

In [ ]:
# orm requires session so rollbacks can occur etc.
session = Session(bind=engine)

In [ ]:
# print out all current column names
for col in processed_df.columns:
    print(f'{col} = ,')

In [ ]:
test = Movie(
id = 34364256,
title = "testing while running",
day = 25,
month = 4,
year = 2020,
date_published = 9-5-2001 ,
user_rating = 5.7 ,
web_rating = 7.6,
vote_num = 345,
reviews_from_users = 456,
reviews_from_critics = 234,
total_reviews = 836,
budget = 987654321,
usa_gross_income = 123456789,
worlwide_gross_income = 1234567891011,
duration = 70,
country = 'USA',
language = "American",
genre = "thriller",
director = " Mr. Jon Dhoe",
writer = "warming",
prod_co = "Universal",
actors = "Brad Pitt, Olando Bloom",
description = "This is a test movie just to make sure all is running well")

In [ ]:
# for index, row in processed_df.iterrows():
#     row['id'] = row['id'].replace("tt", "") 
#     row['worlwide_gross_income'] =  row['worlwide_gross_income'].replace("$", "")
#     row['worlwide_gross_income'] =  row['worlwide_gross_income'].replace(" ", "")
# processed_df

In [ ]:
# add one item
session.add(patterson)

In [ ]:
session.commit()

In [ ]:
session.commit()
engine.execute("select * from cat").fetchall()

In [ ]:
session.close()